# Single Subject Segmentation Q1K

In [ ]:
##use these commented parameters for testing this notebook outside of the automated loop of q1k_automated_reports.ipynb
#subject_id = "100105M1"
#task_id = "VEP"
#session_id = "01"
#run_id = "1"

#use these parameters when executing this notebook from the automation notebook..
subject_id = ""
task_id = ""
session_id = ""
run_id = ""


In [ ]:
# import packages
import mne
import mne_bids
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from pathlib import Path
import shutil
import pylossless as ll
from autoreject import AutoReject

import warnings
warnings.filterwarnings('ignore')


# Set the parameters and read the pylossless data.

In [ ]:
project_path = "/project/def-emayada/q1k/experimental/"
pylossless_path = "derivatives/pylossless/"
sync_loss_path = "derivatives/sync_loss/"
segment_path = "derivatives/segment/"

bids_path = mne_bids.BIDSPath(
    subject=subject_id, session=session_id, task=task_id, run="1", datatype="eeg", suffix="eeg",root=project_path + pylossless_path + sync_loss_path
)
print(bids_path)

In [ ]:
# Read the BIDS pylossless output file..
eeg_raw = mne_bids.read_raw_bids(bids_path=bids_path, verbose=False)

In [ ]:
#show channel types...
channel_types = eeg_raw.get_channel_types()
print("Channel Types:", channel_types)
print("Channel Names:", eeg_raw.info['ch_names'])

In [ ]:
# Define a few channel groups of interest and plot the data
frontal = ["E19", "E11", "E4", "E12", "E5"]
occipital = ["E61", "E62", "E78", "E67", "E72", "E77"]
din = ["DIN"]
pupil = ["pupil_left"]
x_pos = ["xpos_left"]
y_pos = ["ypos_left"]

scale_dict = dict(eeg=1e-4, eyegaze=30, pupil=30)

# picks must be numeric (not string) when passed to `raw.plot(..., order=)`
picks_idx = mne.pick_channels(eeg_raw.ch_names, din + frontal + occipital + pupil + x_pos + y_pos, ordered=True)
eeg_raw.plot(start=0,duration=30,order=picks_idx, scalings=scale_dict)

# Segment the data to 'sv06' and 'sv15'

In [ ]:
# Get the events form the annotations
eeg_events, eeg_event_dict  = mne.events_from_annotations(eeg_raw)

# Relabel condition vars for niceness
eeg_event_dict['sv/visual/disp/sv06'] = eeg_event_dict.pop('sv06')
eeg_event_dict['sv/visual/disp/sv15'] = eeg_event_dict.pop('sv15')
eeg_event_dict['sv/visual/disp/sv06_d'] = eeg_event_dict.pop('sv06_d')
eeg_event_dict['sv/visual/disp/sv15_d'] = eeg_event_dict.pop('sv15_d')

In [ ]:
#reject_criteria = {'eeg': 400e-6}

In [ ]:
# Epoch structure is created for ALL events, then you 'segment' by indexing into it
epochs = mne.Epochs(eeg_raw, eeg_events, event_id=eeg_event_dict, tmin=-1, tmax=2.0, on_missing='warn', event_repeated='drop')
epochs = epochs[['sv06_d', 'sv15_d']]
display(epochs)

#save the cleaned epochs...
epochs.save(project_path + pylossless_path + sync_loss_path + segment_path + 'epoch_fif_files/VEP/' + bids_path.basename + '_epo.fif', overwrite=True)

In [ ]:
# peak... at the EEG channel types
channel_types = epochs.get_channel_types()
print("EEG Channel Types:", channel_types)
print("EEG Channel Names:", epochs.info['ch_names'])

In [ ]:
evokeds = {'sv06_d': epochs['sv06_d'].average(picks=['eeg','misc']), 'sv15_d': epochs['sv15_d'].average(picks=['eeg','misc'])}
mne.write_evokeds(project_path + pylossless_path + sync_loss_path + segment_path + 'erp_fif_files/VEP/' + bids_path.basename + '_erp.fif',list(evokeds.values()), overwrite=True)

# Plot ERP envelopes and topographies

In [ ]:
evokeds['sv06_d'].plot_joint(picks=['eeg'], title='6Hz ERP')

In [ ]:
evokeds['sv15_d'].plot_joint(picks=['eeg'],title='15Hz ERP')

# Plot the ERP overlay

In [ ]:
# Plot ERP overlay
mne.viz.plot_compare_evokeds(evokeds, picks=['E70'], combine='mean')

In [ ]:
# Plot ERP overlay
mne.viz.plot_compare_evokeds(evokeds, picks=['pupil_left'], combine='mean')

In [ ]:
ch_name = 'E70'

decim = 2
freqs = np.arange(3, 50, 2)  # define frequencies of interest
n_cycles = freqs / 2

In [ ]:
decim = 2
freqs = np.arange(2, 50, 2)  # define frequencies of interest
n_cycles = freqs / 2

pow_1, itc_1 = mne.time_frequency.tfr_morlet(
    epochs['sv06_d'],
    freqs,
    picks=ch_name,
    n_cycles=n_cycles,
    decim=decim,
    return_itc=True,
    average=True,
)

pow_2, itc_2 = mne.time_frequency.tfr_morlet(
    epochs['sv15_d'],
    freqs,
    picks=ch_name,
    n_cycles=n_cycles,
    decim=decim,
    return_itc=True,
    average=True,
)

itc_dat_1 = itc_1.data[0, :, :]  # only 1 channel as 3D matrix
pow_dat_1 = pow_1.data[0, :, :]  # only 1 channel as 3D matrix

itc_dat_2 = itc_2.data[0, :, :]  # only 1 channel as 3D matrix
pow_dat_2 = pow_2.data[0, :, :]  # only 1 channel as 3D matrix

In [ ]:
times = 1e3 * epochs['sv06_d'].times  # change unit to ms

fig1, (ax1t, ax1b) = plt.subplots(2, 1, figsize=(6, 4))
fig1.subplots_adjust(0.12, 0.08, 0.96, 0.94, 0.2, 0.43)

ax1t.imshow(
    pow_dat_1,
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    aspect="auto",
    origin="lower",
    cmap="RdBu_r",
)

ax1b.imshow(
    itc_dat_1,
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    aspect="auto",
    origin="lower",
    cmap="RdBu_r",
)

ax1t.set_ylabel("Frequency (Hz)")
ax1t.set_title(f"6Hz Induced power ({ch_name})")
ax1b.set_title(f"6Hz Inter Trial Coherence ({ch_name})")
ax1b.set_xlabel("Time (ms)")

plt.show()

In [ ]:
fig2, (ax2t, ax2b) = plt.subplots(2, 1, figsize=(6, 4))
fig2.subplots_adjust(0.12, 0.08, 0.96, 0.94, 0.2, 0.43)

ax2t.imshow(
    pow_dat_2,
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    aspect="auto",
    origin="lower",
    cmap="RdBu_r",
)

ax2b.imshow(
    itc_dat_2,
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    aspect="auto",
    origin="lower",
    cmap="RdBu_r",
)

ax2t.set_ylabel("Frequency (Hz)")
ax2t.set_title(f"15Hz Induced power ({ch_name})")
ax2b.set_title(f"15Hz Inter Trial Coherence ({ch_name})")
ax2b.set_xlabel("Time (ms)")

plt.show()

In [ ]:
fig3, (ax3t, ax3b) = plt.subplots(2, 1, figsize=(6, 4))
fig3.subplots_adjust(0.12, 0.08, 0.96, 0.94, 0.2, 0.43)

ax3t.imshow(
    pow_dat_2 - pow_dat_1,
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    aspect="auto",
    origin="lower",
    cmap="RdBu_r",
)

ax3b.imshow(
    itc_dat_2 - itc_dat_1,
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    aspect="auto",
    origin="lower",
    cmap="RdBu_r",
)

ax3t.set_ylabel("Frequency (Hz)")
ax3t.set_title(f"15Hz - 6Hz Induced power ({ch_name})")
ax3b.set_title(f"15Hz - 6Hz Inter Trial Coherence ({ch_name})")
ax3b.set_xlabel("Time (ms)")

plt.show()

In [ ]:
#!jupyter nbconvert --output {"session_reports/" + bids_path.basename + ".html"} --TagRemovePreprocessor.remove_all_outputs_tags='{"exclude"}' --no-input --to html session_seg_vp.ipynb